In [1]:
import pandas as pd

# Load the CSV file
file_path = 'aisdk-2024-06-13_10000.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
df.head()


# Timestamp Type of mobile       MMSI   Latitude  Longitude  \
0  13/06/2024 00:00:00        Class A  229077000  57.560762   9.544667   
1  13/06/2024 00:00:00        Class A  209114000  56.957068  10.398268   
2  13/06/2024 00:00:00        Class A  219030044  57.028703   9.943782   
3  13/06/2024 00:00:00        Class A  304244000  91.000000   0.000000   
4  13/06/2024 00:00:00        Class B  219010443  56.987952   9.633877   

      Navigational status  ROT   SOG    COG  Heading  ... Length  \
0  Under way using engine  0.0  17.6   58.1     58.0  ...    NaN   
1  Under way using engine  0.0   8.3  293.8    292.0  ...    NaN   
2           Unknown value  NaN   0.1    0.0      NaN  ...    NaN   
3           Unknown value  NaN   NaN    NaN      NaN  ...   88.0   
4           Unknown value  NaN   0.1   40.3      NaN  ...    NaN   

  Type of position fixing device Draught  Destination                  ETA  \
0                      Undefined     NaN      Unknown                  NaN   
1                      Undefined     NaN      Unknown                  NaN   
2                      Undefined     NaN      Unknown                  NaN   
3                            GPS     5.3  BREMERHAVEN  14/06/2024 06:00:00   
4                      Undefined     NaN      Unknown                  NaN   

   Data source type     A    B    C    D  
0               AIS   NaN  NaN  NaN  NaN  
1               AIS   NaN  NaN  NaN  NaN  
2               AIS   NaN  NaN  NaN  NaN  
3               AIS  82.0  6.0  6.0  7.0  
4               AIS   NaN  NaN  NaN  NaN  

[5 rows x 26 columns]

In [4]:
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import json

# Load the CSV file
file_path = 'aisdk-2024-06-13_10000.csv'
df = pd.read_csv(file_path)

# Function to create nodes and edges for the knowledge graph
def create_knowledge_graph(dataframe):
    G = nx.DiGraph()

    for index, row in dataframe.iterrows():
        # Create a node for each vessel
        vessel_id = row['MMSI']
        G.add_node(vessel_id, **{
            'timestamp': row['# Timestamp'],
            'type_of_mobile': row['Type of mobile'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'navigational_status': row['Navigational status'],
            'ROT': row['ROT'],
            'SOG': row['SOG'],
            'COG': row['COG'],
            'heading': row['Heading'],
            'type_of_position_fixing_device': row['Type of position fixing device'],
            'draught': row['Draught'],
            'destination': row['Destination'],
            'ETA': row['ETA'],
            'data_source_type': row['Data source type'],
            'A': row['A'],
            'B': row['B'],
            'C': row['C'],
            'D': row['D']
        })
        
        # Add edges representing the location relationship
        location_node = f"location_{row['Latitude']}_{row['Longitude']}"
        if location_node not in G:
            G.add_node(location_node, latitude=row['Latitude'], longitude=row['Longitude'])
        
        G.add_edge(vessel_id, location_node, relationship="located at")
    
    return G

# Create the knowledge graph
kg = create_knowledge_graph(df)

# Convert to a JSON serializable format
kg_data = json_graph.node_link_data(kg)

# Save the knowledge graph to a JSON file
with open('knowledge_graph.json', 'w') as f:
    json.dump(kg_data, f, indent=2)

kg_data  # Display the JSON data structure of the knowledge graph


{'directed': True,
 'multigraph': False,
 'graph': {},
 'nodes': [{'timestamp': '13/06/2024 00:00:48',
   'type_of_mobile': 'Class A',
   'latitude': 57.562737,
   'longitude': 9.550602,
   'navigational_status': 'Under way using engine',
   'ROT': 1.1,
   'SOG': 17.6,
   'COG': 58.9,
   'heading': 58.0,
   'type_of_position_fixing_device': 'Undefined',
   'draught': nan,
   'destination': 'Unknown',
   'ETA': nan,
   'data_source_type': 'AIS',
   'A': nan,
   'B': nan,
   'C': nan,
   'D': nan,
   'id': 229077000},
  {'latitude': 57.560762,
   'longitude': 9.544667,
   'id': 'location_57.560762_9.544667'},
  {'timestamp': '13/06/2024 00:00:51',
   'type_of_mobile': 'Class A',
   'latitude': 56.957852,
   'longitude': 10.39504,
   'navigational_status': 'Under way using engine',
   'ROT': 0.0,
   'SOG': 8.3,
   'COG': 293.9,
   'heading': 291.0,
   'type_of_position_fixing_device': 'Undefined',
   'draught': nan,
   'destination': 'Unknown',
   'ETA': nan,
   'data_source_type': 'AIS',